In [9]:
from finance.mongo.extraction import query_mongodb
from finance.params import *
import numpy as np
import pandas as pd


In [47]:
exchange_ls = EXCHANGE_LS
table_name = 'key_ratio'
kwargs = {'calendarYear': { "$gt": 2021 }}

In [48]:
client = PY_MONGO_CLIENT
collection = client['finance'][table_name]

In [49]:
collection.update_many({}, [{'$set': {'calendarYear': {'$toInt': '$calendarYear'}}}])
collection.update_many({}, [{'$set': {'date': {'$toDate': '$date'}}}])

UpdateResult({'n': 3678968, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [50]:
result = query_mongodb(exchange_ls=[], table=table_name, **kwargs)

exchange {'calendarYear': {'$gt': 2021}} - 339470


In [51]:
df  = pd.DataFrame(result)

In [52]:
df.shape

(339470, 60)

In [53]:
df = df.drop(columns='_id').drop_duplicates()


In [17]:
df.shape

(59980, 59)

In [18]:
df.dropna(inplace=True)

In [19]:
df_small = df.select_dtypes(include='float64')
df_small

,currentRatio,quickRatio,cashRatio,daysOfSalesOutstanding,daysOfInventoryOutstanding,operatingCycle,daysOfPayablesOutstanding,cashConversionCycle,grossProfitMargin,operatingProfitMargin,...,priceToSalesRatio,priceEarningsRatio,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio,priceEarningsToGrowthRatio,priceSalesRatio,dividendYield,enterpriseValueMultiple,priceFairValue
0,0.374705,0.342218,0.029178,76.639837,0.000000,76.639837,816.277407,-739.637569,0.711428,-0.207323,...,3.815997,-4.601518,-20.337391,-21.027670,-21.027670,0.000000,3.815997,0.000000,-29.076732,-1.051950
1,1.642388,1.560930,0.357508,108.026195,0.000000,108.026195,19.716626,88.309569,0.719812,-0.032321,...,10.810373,-124.248378,151.809306,149.686187,149.686187,1.471950,10.810373,0.000690,196.201847,21.580184
2,1.599418,1.429121,0.405268,104.454652,0.000000,104.454652,7.899105,96.555546,0.677672,-0.138433,...,13.725316,-21.866842,94.766608,89.688305,89.688305,-0.176346,13.725316,0.000608,-141.676676,28.156364
3,1.675344,1.519580,0.354381,110.017943,0.000000,110.017943,11.547062,98.470881,0.699102,-0.068227,...,12.304106,-48.192957,70.728398,58.799271,58.799271,0.302105,12.304106,0.000618,-352.927421,25.920684
4,1.291686,0.890847,0.203204,27.188659,14.816474,42.005133,32.853696,9.151437,0.121711,0.073884,...,1.346533,6.363925,29.169728,19.881044,19.881044,0.239772,1.346533,0.008711,20.276735,1.731956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119192,1.757204,0.935524,0.342161,55.243577,165.723455,220.967032,35.122068,185.844964,0.625489,0.135896,...,9.797470,31.250000,189.468882,79.030126,79.030126,0.535714,9.797470,0.000000,60.008008,2.974593
119193,1.732013,0.903313,0.357409,56.276571,183.121711,239.398282,37.210688,202.187594,0.618325,0.086854,...,10.950135,51.041667,-910.227446,358.953660,358.953660,0.000000,10.950135,0.000000,79.623783,3.061897
119363,2.464200,1.028467,0.036953,69.052589,105.994989,175.047578,59.606703,115.440875,0.094153,0.021725,...,0.655435,41.043709,-15.761424,130.941064,130.941064,-0.492525,0.655435,0.000000,19.101905,0.492912
119364,2.445956,0.942038,0.001395,61.115825,108.454797,169.570622,62.286696,107.283926,0.131323,0.056256,...,0.366274,2.226500,27.244150,7.611691,7.611691,0.022265,0.366274,0.000000,10.078868,0.289220


In [20]:
from sklearn.preprocessing import StandardScaler

df_small_ss = StandardScaler().fit_transform(df_small)

In [21]:
from sklearn.decomposition import PCA

In [22]:
pca = PCA(26)
pca.fit(df_small_ss)

PCA(n_components=26)

In [23]:
for i, pc in enumerate(np.cumsum(pca.explained_variance_ratio_)):
    print(i, pc)

0 0.1318615721567038
1 0.21861625008890398
2 0.29211963637673405
3 0.34781559451833133
4 0.40337128987741155
5 0.45883232515762123
6 0.5113467533005938
7 0.5535728076613939
8 0.5917690848475272
9 0.6288061243133236
10 0.6644662381334929
11 0.6903403454768519
12 0.7115817926625179
13 0.732401157637744
14 0.7518050849254523
15 0.7704719307305228
16 0.789017243152787
17 0.8075426919701575
18 0.8260634654572642
19 0.8445817247998733
20 0.8630933076840279
21 0.8816016794422257
22 0.9000687684778808
23 0.9177017556666354
24 0.935056791358446
25 0.9510394028097314


In [24]:
pca = PCA(26)
pca.fit(df_small_ss)

PCA(n_components=26)

In [25]:
new = pca.fit_transform(df_small)

In [26]:
from sklearn.metrics.pairwise import cosine_similarity
from umap import UMAP
from hdbscan import HDBSCAN

/Users/michieldekoninck/.pyenv/versions/3.10.6/envs/finance/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
umap_model = UMAP(n_neighbors=10, n_components=5, min_dist=0.0, metric='cosine')
new_umap = umap_model.fit_transform(df_small_ss)

In [28]:
hdbscan_model = HDBSCAN(
    min_cluster_size=30,
    metric='euclidean',
    prediction_data=False)

new_cluster = hdbscan_model.fit(new_umap)
df['cluster'] = new_cluster.labels_

In [29]:
df.cluster.value_counts()

cluster
-1      7306
 19     6090
 3      1672
 5      1139
 29      879
        ... 
 145      32
 109      32
 127      31
 112      31
 125      31
Name: count, Length: 147, dtype: int64

In [30]:
df[df['symbol']=='NVDA']

,symbol,date,calendarYear,period,currentRatio,quickRatio,cashRatio,daysOfSalesOutstanding,daysOfInventoryOutstanding,operatingCycle,...,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio,priceEarningsToGrowthRatio,priceSalesRatio,dividendYield,enterpriseValueMultiple,priceFairValue,key,cluster
72986,NVDA,2023-10-29,2024,Q3,3.588397,3.528074,0.606417,41.269868,91.125000,132.394868,...,144.010984,138.550666,138.550666,0.549526,56.062554,0.000096,93.194440,18.760683,2024Q3NVDA,53
72987,NVDA,2023-07-30,2024,Q2,2.786627,2.234275,0.559609,47.082254,96.096415,143.178669,...,190.725890,182.042875,182.042875,0.232040,85.556243,0.000086,156.601156,42.020587,2024Q2NVDA,53
72988,NVDA,2023-04-30,2024,Q1,3.427410,2.672176,0.699587,51.056730,163.125000,214.181730,...,268.147578,245.302989,245.302989,1.915659,99.287681,0.000139,271.113577,29.122227,2024Q1NVDA,-1
72989,NVDA,2023-01-29,2023,Q4,3.515618,2.609020,0.516380,56.921170,209.337241,266.258411,...,271.351540,209.938497,209.938497,0.751302,78.028703,0.000208,270.009933,21.363363,2023Q4NVDA,-1
72990,NVDA,2022-10-30,2023,Q3,3.387746,2.633260,0.408461,74.476480,145.555556,220.032035,...,-2428.481957,854.924770,854.924770,32.034534,56.504891,0.000298,317.415600,15.697715,2023Q3NVDA,36
72991,NVDA,2022-07-31,2023,Q2,3.620494,2.951802,0.397861,71.379773,92.375297,163.755070,...,549.704839,362.285787,362.285787,-2.922402,68.631108,0.000217,510.655719,19.290719,2023Q2NVDA,93
72992,NVDA,2022-05-01,2023,Q1,5.317332,4.634304,0.698849,59.051641,99.639482,158.691123,...,357.297066,282.782773,282.782773,-1.650175,59.060929,0.000204,139.693534,18.597910,2023Q1NVDA,-1


In [31]:
df[df['cluster']==55].sort_values('priceToFreeCashFlowsRatio').tail()

,symbol,date,calendarYear,period,currentRatio,quickRatio,cashRatio,daysOfSalesOutstanding,daysOfInventoryOutstanding,operatingCycle,...,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio,priceEarningsToGrowthRatio,priceSalesRatio,dividendYield,enterpriseValueMultiple,priceFairValue,key,cluster
7259,CRWD,2022-07-31,2023,Q2,1.804444,1.659941,1.406008,70.432026,94.839116,165.271142,...,313.995950,203.077321,203.077321,-3.584393,79.654133,0.000000,-2686.561193,34.785774,2023Q2CRWD,55
10095,SWAV,2023-06-30,2023,Q2,5.688881,4.427213,1.763376,48.236783,338.397501,386.634284,...,337.670792,263.333421,263.333421,-3.463701,58.080538,0.000000,307.022983,16.962030,2023Q2SWAV,55
12344,ISSC,2023-03-31,2023,Q2,8.742756,6.732350,5.570158,50.629691,204.201260,254.830952,...,365.873851,321.997663,321.997663,0.334004,17.351267,0.000000,71.375416,3.730142,2023Q2ISSC,55
6887,PSK.TO,2023-03-31,2023,Q1,0.800283,0.793201,-0.004958,80.436162,0.000000,80.436162,...,433.259322,297.236047,297.236047,-1.575142,40.542902,0.011208,47.635607,1.847120,2023Q1PSK.TO,55
100301,PREKF,2023-03-31,2023,Q1,0.800283,0.793201,-0.004958,80.436162,0.000000,80.436162,...,434.185230,297.871262,297.871262,-1.578508,40.629546,0.011184,47.732380,1.851068,2023Q1PREKF,55


In [32]:
moats = ['symbol', 'operatingProfitMargin', 'date', 'freeCashFlowOperatingCashFlowRatio', 'netProfitMargin', 'returnOnAssets', 'returnOnEquity']
health = ['currentRatio', 'debtEquityRatio', 'receivablesTurnover']

In [33]:
moat = pd.DataFrame()

moat['symbol'] = df['symbol']
moat['date'] = df['date']
moat['operatingmargin'] = df['operatingProfitMargin'].apply(lambda x: x/0.15)
moat['fcfmargin'] = df['freeCashFlowOperatingCashFlowRatio'].apply(lambda x: x/0.05)
moat['netmargin'] = df['netProfitMargin'].apply(lambda x: x/0.15)
moat['roa'] = df['returnOnAssets'].apply(lambda x: x/0.06)
moat['roe'] = df['returnOnEquity'].apply(lambda x: x/0.01)

In [34]:
df[moats]

,symbol,operatingProfitMargin,date,freeCashFlowOperatingCashFlowRatio,netProfitMargin,returnOnAssets,returnOnEquity
0,MYID.V,-0.207323,2023-03-31,1.033941,-0.207323,-0.195103,0.057152
1,PEGA,-0.032321,2023-09-30,0.986015,-0.021752,-0.005954,-0.043421
2,PEGA,-0.138433,2023-06-30,0.946413,-0.156919,-0.039251,-0.321907
3,PEGA,-0.068227,2023-03-31,0.831339,-0.063827,-0.016206,-0.134463
4,PSX,0.073884,2023-09-30,0.681564,0.052897,0.026959,0.068038
...,...,...,...,...,...,...,...
119192,OSSFF,0.135896,2023-06-30,0.417114,0.078380,0.011711,0.023797
119193,OSSFF,0.086854,2023-03-31,-0.394356,0.053633,0.007272,0.014997
119363,OBGRF,0.021725,2023-03-31,-8.307692,0.003992,0.001405,0.003002
119364,OBGRF,0.056256,2022-12-31,0.279388,0.041127,0.015429,0.032475


In [46]:
moat.groupby(['symbol', 'date']).mean()

operatingmargin  fcfmargin  netmargin       roa  \
symbol  date                                                          
0038.HK 2023-03-31         0.960623  18.510061   0.675429  0.571312   
        2023-06-30         0.709426  19.417930   0.585886  0.314919   
        2023-09-30         0.751850  18.961902   0.734868  0.374253   
0081.HK 2023-03-31         0.803093   0.000000   0.421770  0.000000   
        2023-06-30         0.803093   0.000000   0.421770  0.090073   
...                             ...        ...        ...       ...   
ZYXI    2023-06-30         0.402355  14.331984   0.497419  0.366392   
        2023-09-30         0.732579  19.469245   0.480016  0.395565   
ZZZ.TO  2023-03-31         0.691252   0.773250   0.365788  0.181559   
        2023-06-30         0.741931  13.835963   0.389351  0.194159   
        2023-09-30         1.087998  16.390965   0.643993  0.377736   

                          roe  cluster  
symbol  date                            
0038.HK 2023-03-31   7.536941     44.0  
        2023-06-30   4.154797     51.0  
        2023-09-30   4.874512     45.0  
0081.HK 2023-03-31  14.213281     20.0  
        2023-06-30   3.030876     20.0  
...                       ...      ...  
ZYXI    2023-06-30   5.386481     41.0  
        2023-09-30   6.134990     51.0  
ZZZ.TO  2023-03-31   2.625882     22.0  
        2023-06-30   3.011397     41.0  
        2023-09-30   5.631809     41.0  

[32609 rows x 6 columns]

In [35]:
ss = StandardScaler()

moat_ss = ss.fit_transform(moat.drop(columns=['symbol', 'date']))

pca = PCA(4)

moat_pca = pca.fit_transform(moat_ss)

pca.explained_variance_ratio_

array([0.47639868, 0.20010354, 0.19988554, 0.08725234])

In [36]:
hdbscan_model = HDBSCAN(
    min_cluster_size=100,
    metric='euclidean',
    prediction_data=False,
    min_samples=15)

new_cluster = hdbscan_model.fit(moat_pca)
moat['cluster'] = new_cluster.labels_

In [37]:
moat['cluster'] .value_counts()

cluster
-1     8794
 41    4840
 20    3800
 53    2808
 40    1100
 16     880
 25     593
 44     503
 30     468
 24     396
 23     335
 37     321
 39     314
 26     312
 3      272
 8      269
 15     264
 45     264
 17     259
 11     259
 51     239
 42     238
 31     226
 49     208
 34     206
 52     206
 19     192
 22     190
 6      189
 29     183
 36     183
 47     173
 7      169
 27     166
 13     161
 9      159
 4      158
 38     157
 1      155
 2      151
 43     147
 18     142
 46     140
 5      140
 12     139
 10     129
 33     128
 48     123
 35     116
 21     110
 14     110
 28     109
 0      107
 50     105
 32     105
Name: count, dtype: int64

In [38]:
moat[moat['symbol']=='NVDA']

,symbol,date,operatingmargin,fcfmargin,netmargin,roa,roe,cluster
72986,NVDA,2023-10-29,3.832597,19.241680,3.400662,2.844980,17.069883,-1
72987,NVDA,2023-07-30,3.356284,19.089477,3.054219,2.081189,22.501000,-1
72988,NVDA,2023-04-30,1.983686,18.296118,1.893771,0.765857,8.331974,44
72989,NVDA,2023-01-29,1.384895,15.473544,1.557869,0.572256,6.397901,41
72990,NVDA,2022-10-30,0.675547,-7.040816,0.764346,0.279918,3.185161,11
72991,NVDA,2022-07-31,0.496221,13.181102,0.652347,0.251480,2.750409,40
72992,NVDA,2022-05-01,1.502574,15.829001,1.301480,0.596449,6.147416,41


In [42]:
moat[moat['cluster']==44].groupby('symbol').count().sort_values('date').tail(40)

,date,operatingmargin,fcfmargin,netmargin,roa,roe,cluster
symbol,,,,,,,
NA9.F,2,2,2,2,2,2,2
EVO.AX,2,2,2,2,2,2,2
UVV,2,2,2,2,2,2,2
NGRRF,2,2,2,2,2,2,2
GOLF,2,2,2,2,2,2,2
WIT,2,2,2,2,2,2,2
G,2,2,2,2,2,2,2
CCC3.DE,2,2,2,2,2,2,2
TRS.AX,2,2,2,2,2,2,2


In [40]:
df[moats].netProfitMargin

0        -0.207323
1        -0.021752
2        -0.156919
3        -0.063827
4         0.052897
            ...   
119192    0.078380
119193    0.053633
119363    0.003992
119364    0.041127
119365    0.020230
Name: netProfitMargin, Length: 32610, dtype: float64